In [76]:
%reload_ext autoreload
%autoreload 2

In [77]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [78]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [79]:
loader = SlackDataLoader("../src/anonymized/")
channel = loader.get_channels()
all_data = []
all_messages = []
for i in channel:
    data = loader.slack_parser(f"../src/anonymized/{i['name']}/")
    all_data.append(data)

for j in channel:
    messages = loader.slack_parser(f"../src/anonymized/{j['name']}/")
    all_messages.append(messages)

# print(all_messages)
df = pd.concat(all_data, ignore_index=True)
# d

In [5]:
# Who are the top 10  users by replay count?
user_reply_counts = df.groupby('sender_name')['reply_count'].sum().sort_values(ascending=False)
top_10_users = user_reply_counts.head(10)
print(top_10_users)

sender_name
Cristian Wilson     620
Vanessa Norman      462
Brian Odom          367
Brenda Hernandez    307
Michelle Lewis      306
April Lucas         302
Travis Butler       299
Amy Leon            282
Joshua Rhodes       247
Brady Rhodes        242
Name: reply_count, dtype: int64


In [80]:
# Who are the bottom 10  users by replay count?
user_reply_counts = df.groupby('sender_name')['reply_count'].sum().sort_values(ascending=False)
bottom_10_users = user_reply_counts.tail(10)
print(bottom_10_users)

sender_name
Randy Miller         4
Leah Newton          2
Blake Scott          0
Patrick Hernandez    0
Patricia Thomas      0
Slackbot             0
Christopher Case     0
Holly Whitaker       0
Erica Munoz          0
David Scott          0
Name: reply_count, dtype: int64


In [122]:
# Who are the top 10  users by mention count?
mentioned_users = utils.get_tagged_users(df)

# loader.get_users()
# users_map = loader.get_user_map()[0]

counts = pd.Series(mentioned_users).value_counts()
top_10_users = counts.head(10)
# print(top_10_users)

Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>
Invalid user ID type: <class 'list'>


In [123]:
# Who are the bottom 10  users by mention count?
bottom_10_users = counts.tail(10)
print(bottom_10_users)

msg_content
[@U03UJGP0C68, @U03UG32J3PC, @U03U9EJR362]                                1
[@U03V6HMRPGQ, @U03TT5KEYCF]                                              1
[@U03UJGP0C68, @U03UUR571A5, @U03UG4Q7V42]                                1
[@U03TT5KEYCF, @U03U4GULU3Y, @U03UL5LSTG9]                                1
[@U03UP7V9Q57, @U03V8LHPDME, @U03UKL27B0R, @U03U9DB7REG, @U03UUN8M4RX]    1
[@U03V1AM5TFA, @U03T89ACUUW, @U03TEPYRM2P, @U03UKL27B0R, @U03V8LHPDME]    1
[@U03UKL27B0R, @U03UP7V9Q57]                                              1
[@U03UH397319, @U03UJGP0C68]                                              1
[@U03UD68RQH3, @U03TEPYRM2P]                                              1
[@U03V61VGQG0, @U03UUR571A5]                                              1
Name: count, dtype: int64


In [83]:
# Who are the top users by Message count?
user_message_counts = df['sender_name'].value_counts()
top_10_users = user_message_counts.head(10)
print(top_10_users)

sender_name
Vanessa Norman      1288
Amy Leon            1150
Travis Butler       1075
April Lucas          991
Brenda Hernandez     843
Cristian Wilson      831
Phillip Atkins       754
Brady Rhodes         641
Judith Bolton        574
Joshua Rhodes        519
Name: count, dtype: int64


In [84]:
# Who are the bottom users by Message count?
bottom_10_users = user_message_counts.tail(10)
print(bottom_10_users)

sender_name
Leah Newton          10
Patricia Thomas       8
Randy Miller          7
Patrick Hernandez     5
Slackbot              3
Erica Munoz           1
Christopher Case      1
Holly Whitaker        1
David Scott           1
Blake Scott           1
Name: count, dtype: int64


In [68]:
# Who are the top users by Reaction count?


In [ ]:
# Who are the bottom users by Reaction count?

In [69]:
# What are the top 10 messages by Replies?
# processes = utils.process_msgs(all)
# print(processes)

In [70]:
# message_list = utils.get_msgs_df_info(df)
# print(message_list)

In [2]:
def map_userid_2_realname(user_profile: dict, comm_dict: dict, plot=False):
    """
    map slack_id to realnames
    user_profile: a dictionary that contains users info such as real_names
    comm_dict: a dictionary that contains slack_id and total_message sent by that slack_id
    """
    user_dict = {} # to store the id
    real_name = [] # to store the real name
    ac_comm_dict = {} # to store the mapping
    count = 0
    # collect all the real names
    for i in range(len(user_profile['profile'])):
        real_name.append(dict(user_profile['profile'])[i]['real_name'])

    # loop the slack ids
    for i in user_profile['id']:
        user_dict[i] = real_name[count]
        count += 1

    # to store mapping
    for i in comm_dict:
        if i in user_dict:
            ac_comm_dict[user_dict[i]] = comm_dict[i]

    ac_comm_dict = pd.DataFrame(data= zip(ac_comm_dict.keys(), ac_comm_dict.values()),
    columns=['LearnerName', '# of Msg sent in Threads']).sort_values(by='# of Msg sent in Threads', ascending=False)
    
    if plot:
        ac_comm_dict.plot.bar(figsize=(15, 7.5), x='LearnerName', y='# of Msg sent in Threads')
        plt.title('Student based on Message sent in thread', size=20)
        
    return ac_comm_dict

In [38]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [54]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?